# **5. (각 10점) (Keras 이용) MNIST 데이터는 60,000개의 훈련 자료와 10,000개의 시험 자료로 이루어져 있다. MNIST 데이터를 이용하여 DNN과 CNN을 비교하려고 한다. 다음 물음에 답하시오.**

## *(a) 아래 왼쪽 그림과 같은 DNN을 디자인하시오. 그리고 해당 모형에 대해 gradient descent algorithm으로 추정해야 하는 모수의 수를 구하시오.*

### 1) 환경설정

In [2]:
#필요한 모듈 불러오기
from tensorflow.keras.datasets import mnist     #MNIST 데이터셋을 keras에서 불러오기
from tensorflow.keras.utils import to_categorical       #분류문제 해결 위한 모듈
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten      #모델 학습 위해 필요한 모듈
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow import keras

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#디렉토리 위치 지정
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 2) 데이터 확인


In [ ]:
#mnist 데이터 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()  #훈련 자료(train data), 시험 자료(test data) 가져오기

In [4]:
#훈련 자료, 평가 자료 분리 
np.random.seed(1234)
val_ind = np.random.choice(x_train.shape[0], 10000, replace=False)  #평가자료를 x_train 데이터에서 10000개 랜덤 샘플링
tr_ind = np.setdiff1d(np.arange(x_train.shape[0]),val_ind)          #원본 x_train과 평가자료의 차집합(나머지 50000개)을 훈련자료로 지정

x_valid, y_valid = x_train[val_ind], y_train[val_ind]
x_train, y_train = x_train[tr_ind], y_train[tr_ind]

### 3) 전처리

In [5]:
#이미지 데이터 전처리
x_train = x_train.astype('float32')     #데이터 타입을 float32로 변경
x_train = x_train/255                   #정규화

x_test = x_test.astype('float32')
x_test = x_test/255

In [6]:
#레이블 데이터 전처리
y_onehot_train = to_categorical(y_train, num_classes=10) # 분류 문제이므로 one-hot encoding을 하여 10개의 레이블이 0, 1로 표현되도록 변경
y_onehot_test = to_categorical(y_test, num_classes=10)

## 4) Neural networks 디자인

### (1) DNN 디자인

In [7]:
#신경망 모델에 들어가는 hyper parameter 설정
OUTPUT_SHAPE = 10                   #output layer의 nod의 개수(출력 데이터가 나오는 포맷) 
BATCH_SIZE = 128                     #mini-batch 사이즈 즉, 학습할 데이터 일부의 개수
EPOCHS = 10           
VERBOSE = 1                          #학습 진행 상황 출력 모드 설정

In [8]:
#신경망 모델 생성 
#신경망 모델 생성 
model = Sequential()
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))


### (2) 추정해야 하는 모수의 수


: 28*28=784개 

## (b) 아래 오른쪽 그림과 같은 CNN을 디자인하시오. 그리고 해당 모형에 대해 gradient descent algorithm으로 추정해야 하는 모수의 수를 구하시오. 

### (1) CNN 디자인

In [9]:
#신경망 모델에 들어가는 hyper parameter 설정
OUTPUT_SHAPE = 10                   #output layer의 nod의 개수(출력 데이터가 나오는 포맷) 
BATCH_SIZE = 128                     #mini-batch 사이즈 즉, 학습할 데이터 일부의 개수
EPOCHS = 10           
VERBOSE = 1       

In [10]:
##데이터 전처리
#파라미터 정규화: -0.5 ~ 0.5 사이의 값을 갖도록 scaling
x_train = (x_train / 255) - 0.5
x_valid = (x_valid / 255) - 0.5
x_test = (x_test / 255) - 0.5

#3차원 tensor로 변환
x_train = np.expand_dims(x_train, axis=3)
x_valid = np.expand_dims(x_valid, axis=3)
x_test = np.expand_dims(x_test, axis=3)

#binary class matrix로 변경 
y_train_onehot = to_categorical(y_train, num_classes=10)
y_valid_onehot = to_categorical(y_valid, num_classes=10)
y_test_onehot = to_categorical(y_test, num_classes=10)

#데이터 전처리 결과 확인
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)
(10000, 28, 28, 1)


In [11]:
##CNN 디자인
#신경망 모델 생성 
filter_size = 3              #convolution 한 층의 filter 및 pooling 할 size 설정
pool_size = 2

model2 = Sequential([
  keras.Input(shape=(28,28,1)),

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'), 
 BatchNormalization(),                                     
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),
   Conv2D(filters = 50, kernel_size = (3,3), padding='same'),   
 BatchNormalization(),                                     
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),

  Flatten(),
  Dense(20),                       
  Activation('relu'),
  Dense(10, activation='softmax'),
])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 50)        500       
                                                                 
 batch_normalization_3 (Batc  (None, 28, 28, 50)       200       
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 28, 28, 50)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 50)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        22550     
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 50)      

### (2) 추정해야하는 모수의 수


=3 * 3 * 50(필터 개수) = 450개

## (c) 60,000개의 훈련 자료를 활용하여 (a), (b)에서 디자인한 모형들을 학습하시오. 단, 두 모형 모두 optimizer는 adam, mini-batch는 128을 이용하여 총 10epoch을 학습하시오 (최적의 모형은 찾지 않아도 된다.).

## (1) DNN 학습 

In [12]:
#신경망 모델 컴파일
model.compile(
    optimizer='adam',                 #최적화 알고리즘 adam 사용
    loss='categorical_crossentropy',  #분류 문제이므로 목적 함수는 cross-entropy 사용
    metrics=['accuracy']              #매 에폭이 끝날 때마다 정분류율 출력
)

In [13]:
history = model.fit(               
    x_train, y_train_onehot,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=VERBOSE,
    validation_split=0
)

Epoch 1/10
391/391 [==============================] - 5s 5ms/step - loss: 0.3311 - accuracy: 0.9004
Epoch 2/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1447 - accuracy: 0.9560
Epoch 3/10
391/391 [==============================] - 2s 5ms/step - loss: 0.1071 - accuracy: 0.9669
Epoch 4/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0860 - accuracy: 0.9740
Epoch 5/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0733 - accuracy: 0.9775
Epoch 6/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0614 - accuracy: 0.9801
Epoch 7/10
391/391 [==============================] - 2s 6ms/step - loss: 0.0649 - accuracy: 0.9783
Epoch 8/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0573 - accuracy: 0.9809
Epoch 9/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0461 - accuracy: 0.9848
Epoch 10/10
391/391 [==============================] - 2s 5ms/step - loss: 0.0527 - accuracy: 0.9831

## (2) CNN 학습

In [14]:
##신경망 모델 컴파일
model2.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [15]:
##신경망 모델 학습
history = model2.fit(x=x_train,y=y_train_onehot, 
          epochs=10, 
          validation_data=(x_valid, y_valid_onehot),
          verbose=1,
          batch_size=128 )

Epoch 1/10
391/391 [==============================] - 10s 9ms/step - loss: 2.3038 - accuracy: 0.1094 - val_loss: 2.4108 - val_accuracy: 0.1150
Epoch 2/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3014 - accuracy: 0.1115 - val_loss: 3.9989 - val_accuracy: 0.1506
Epoch 3/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3013 - accuracy: 0.1115 - val_loss: 4.3015 - val_accuracy: 0.1534
Epoch 4/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3013 - accuracy: 0.1115 - val_loss: 4.3114 - val_accuracy: 0.1532
Epoch 5/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3013 - accuracy: 0.1115 - val_loss: 4.3073 - val_accuracy: 0.1536
Epoch 6/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3013 - accuracy: 0.1115 - val_loss: 4.3057 - val_accuracy: 0.1544
Epoch 7/10
391/391 [==============================] - 3s 8ms/step - loss: 2.3013 - accuracy: 0.1115 - val_loss: 4.3128 - val_accuracy: 0.1538
Epoch

## (d) (c)를 통해 10epoch 동안 학습한 후의 (a) 모형과 (b) 모형의 시험 자료에서의 정분류율을 계산하시오. 어떤 모형이 더 높은 정분류율을 갖는가? 각 모형의 모수의 개수를 고려할 때, 이미지 분석에 있어서 어떤 모형이 더 우수하다고 할 수 있는지 논의하시오.

### (1) (a) 모형의 시험자료에서의 정분류율


In [16]:
model.evaluate(x_test, y_test_onehot) 

313/313 [==============================] - 1s 3ms/step - loss: 442.2672 - accuracy: 0.0889


[442.2672119140625, 0.08889999985694885]

### (2) (b) 모형의 시험자료에서의 정분류율

In [17]:
model2.evaluate(x_test, y_test_onehot) 

313/313 [==============================] - 1s 3ms/step - loss: 2.3010 - accuracy: 0.1135


[2.3010458946228027, 0.11349999904632568]

### (3) 이미지 분석에 있어 어떤 모형이 우수한지

: CNN 모형에서의 정분류율(b모형에서의 정분류율)이 더 높다. 또한, 추정해야 하는 모수의 개수도 CNN 모형이 훨씬 적으며, 이는 모형의 성능 및 계산 속도 개선과도 직결되는 문제이다. 따라서 이미지 분석에 있어 CNN 모형이 우수하다.